In [1]:
import sys
from glob import glob

import h5py
import pygrib
import numpy as np
from datetime import datetime, timedelta

In [2]:
import zarr
import xarray

In [6]:
sys.path.insert(0, '/glade/u/home/ksha/NCAR/')
sys.path.insert(0, '/glade/u/home/ksha/NCAR/libs/')

from namelist import *
import data_utils as du

In [7]:
var_names = [
    'Maximum/Composite radar reflectivity',
    'MSLP (MAPS System Reduction)',
    '2 metre temperature',
    '2 metre specific humidity',
    '2 metre dewpoint temperature',
    '2 metre relative humidity',
    '10 metre U wind component',
    '10 metre V wind component',
    'Total Precipitation',
    'Convective available potential energy',
    'Convective inhibition',
    'Precipitable water',
    'Storm relative helicity',
    'Storm relative helicity',
    'U-component storm motion',
    'V-component storm motion',
    'Vertical u-component shear',
    'Vertical v-component shear',
    'Vertical u-component shear',
    'Vertical v-component shear']

var_inds = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]

In [8]:
var_save=['max_radar', 'mslp', 't2m', 'q2m', 'dp2m', 'rh2m', 'u10', 'v10', 
            'apcp', 'cape', 'cin', 'pwat', 'storm_heli_3k', 'storm_heli_1k', 
            'u_storm', 'v_storm', 'u_shear_1k', 'v_shear_1k', 'u_shear_6k', 'v_shear_6k']

### 12-hr lead time

In [18]:
filenames = sorted(glob(HRRR_dir+'fcst12hr/*.grib2'))
filenames = filenames[:2]

In [19]:
with pygrib.open(filenames[0]) as grbio:
    var = grbio.select(name='Land-sea mask')[0]
    land_mask = var.values
    lat_hrrr, lon_hrrr = var.latlons() 

In [20]:
grid_shape = lat_hrrr.shape

In [21]:
N_files = len(filenames[:2])
N_vars = len(var_names)
out = np.empty((N_files,)+grid_shape+(N_vars,))

In [22]:
for n, filename in enumerate(filenames):
    with pygrib.open(filename) as grbio:
        for i, name in enumerate(var_names):
            var = grbio.select(name=name)[var_inds[i]]
            out[n, ..., i] = var.values

In [32]:
# var_inds = [1003, 1014, 1041, 1042, 1043, 1044, 1045, 1047, 1048, 
#             1060, 1074, 1075, 1076, 1097, 1098, 1099, 1100, 1101, 1102, 1103, 1104]

In [27]:
out_name = 'hrrr12hr.zarr'

In [28]:
zarr.save(HRRR_dir+out_name, out)

In [29]:
test = zarr.load(HRRR_dir+out_name)

In [30]:
test.shape

(2, 1059, 1799, 20)

In [23]:
out.shape

(2, 1059, 1799, 20)